## 一. 词向量
#### 自然语言理解的问题要转化为机器学习的问题，第一步肯定是要找一种方法把这些符号数学化。
1. NLP 中最直观，也是到目前为止最常用的词表示方法是 One-hot Representation，这种方法把每个词表示为一个很长的向量。这个向量的维度是词表大小，其中绝大多数元素为 0，只有一个维度的值为 1，这个维度就代表了当前的词。这种 One-hot Representation 如果采用稀疏方式存储，会是非常的简洁：也就是给每个词分配一个数字 ID。比如刚才的例子中，话筒记为 3，麦克记为 8（假设从 0 开始记）。如果要编程实现的话，用 Hash 表给每个词分配一个编号就可以了。这么简洁的表示方法配合上最大熵、SVM、CRF 等等算法已经很好地完成了 NLP 领域的各种主流任务。<font color=red>one-hot表示最大的问题就是‘语义鸿沟’任意两词都是孤立的。</font> 
2. Deep Learning 中一般用到的词向量并不是刚才提到的用 One-hot Representation 表示的那种很长很长的词向量，而是用 Distributed Representation（不知道这个应该怎么翻译，因为还存在一种叫“Distributional Representation”的表示方法，又是另一个不同的概念）表示的一种低维实数向量。这种向量一般长成这个样子：[0.792, −0.177, −0.107, 0.109, −0.542, ...]。维度以 50 维和 100 维比较常见。Distributed representation 最大的贡献就是让相关或者相似的词，在距离上更接近了。向量的距离可以用最传统的欧氏距离来衡量，也可以用 cos 夹角来衡量。用这种方式表示的向量，“麦克”和“话筒”的距离会远远小于“麦克”和“天气”。<font color=red>Distributed representation 用来表示词，通常被称为“Word Representation”或“Word Embedding”，中文俗称“词向量”。</font>



## 二.语言模型
#### 语言模型形式化的描述就是给定一个字符串，看它是自然语言的概率 P(w1,w2,…,wt)。w1 到 wt 依次表示这句话中的各个词：P(w1,w2,…,wt)=P(w1)×P(w2|w1)×P(w3|w1,w2)×…×P(wt|w1,w2,…,wt−1)
### N-gram语言模型
1. <font color=red>N-gram语言模型</font>：为了简化问题，可以做一个n-1阶Markov假设，即每个单词只和其前n-1个单词相关，则 P(wt|wt−n+1,…,wt−1)，这样，只要算出这些条件概率，任意的句子的概率我们都可以计算，而这些条件概率可以通过对语料建立一个多项分布模型用最大似然法求得。

### 神经网络语言模型
2. <font color=red>神经网络语言模型</font> 传统N-gram语言模型的缺点：1.n-gram语言模型无法建模更远的关系，语料的不足使得无法训练更高阶的语言模型。2.这种模型无法建模出词之间的相似度。既然我们想要的就是P(wt|Wp)，那么可以用一个神经网络直接去建模这个概率，网络的输入是前N-1单词，输出是V个节点，其中第i个节点的输出值就是P(wi|Wp).这里用Wp表示wt前面的N-1的个词。
![](https://github.com/xx674967/githubdesktop/blob/12/pic/11.30/1.png?raw=true)
![](https://github.com/xx674967/githubdesktop/blob/12/pic/11.30/3.jpg?raw=true)

### <font color=red>上式中输入$X$的优化参数矩阵C，就是我们需要的词向量转化矩阵（word embedding matrix）。</font>


## 三.Word2vec
#### 神经网络语言模型解决了概率平滑、数据稀疏及维度灾难等问题，但它仍然无法彻底解决n的问题，即通过前n个词来预测接着的单词，这里认为神经网络语言模型无法取n大于5。另外一个是训练需要花费很多时间。Word2vec是google的研究人员发布了一个工具包叫word2vec，利用神经网络为单词寻找一个连续向量空间中的表示。包括CBOW模型和Skip-gram模型，如下图。另外CBOW模型和Skip-gram模型的输入向量中不再使用one-hot形式，而是可以假设一个m维向量，初始值可以随机设置，然后通过训练不断优化最后得到具体的向量值。这也就是为什么说我们在训练CBOW模型和Skip-gram模型时会产生一个词向量的副产品，该词向量是一个分布式词向量。
![](https://github.com/xx674967/githubdesktop/blob/12/pic/11.30/2.jpg?raw=true)
#### CBOW（Continuous Bag-of-Words）模型和 Skip-Gram模型
1.CBOW模型的训练输入是某一个特征词的上下文相关的词对应的词向量，而输出就是这特定的一个词的词向量。比如下面这段话，我们的上下文大小取值为4，特定的这个词是"Learning"，也就是我们需要的输出词向量,上下文对应的词有8个，前后各4个，这8个词是我们模型的输入。由于CBOW使用的是词袋模型，因此这8个词都是平等的，也就是不考虑他们和我们关注的词之间的距离大小，只要在我们上下文之内即可。
比如我们的输入是8个词向量（上文四个下文四个），输出是所有词的softmax概率（训练的目标是期望训练样本特定词对应的softmax概率最大），对应的CBOW神经网络模型输入层有8个神经元，输出层有词汇表大小个神经元。隐藏层的神经元个数我们可以自己指定。通过DNN的反向传播算法，我们可以求出DNN模型的参数，同时得到所有的词对应的词向量。这样当我们有新的需求，要求出某8个词对应的最可能的输出中心词时，我们可以通过一次DNN前向传播算法并通过softmax激活函数找到概率最大的词对应的神经元即可。

2.Skip-Gram模型和CBOW的思路是反着来的，即输入是特定的一个词的词向量，而输出是特定词对应的上下文词向量。比如我们的上下文大小取值为4， 特定的这个词"Learning"是我们的输入，而这8个上下文词是我们的输出。
这样我们这个Skip-Gram的例子里，我们的输入是特定词， 输出是softmax概率排前8的8个词，对应的Skip-Gram神经网络模型输入层有1个神经元，输出层有词汇表大小个神经元。隐藏层的神经元个数我们可以自己指定。通过DNN的反向传播算法，我们可以求出DNN模型的参数，同时得到所有的词对应的词向量。这样当我们有新的需求，要求出某1个词对应的最可能的8个上下文词时，我们可以通过一次DNN前向传播算法得到概率大小排前8的softmax概率对应的神经元所对应的词即可。
#### 对于CBOW和Skip-gram两个模型，word2vec给出了两套框架，他们分别基于Hierarchical Softmax和Negative Sampling。基于Hierarchical Softmax，首先需要了解Huffman编码和Huffman树。
·在通信领域，根据实际字符出现的频率编码，频率低的字符用长码，频率高的字符用短码，这样能最大限度的优化编码。可以用字符的作为节点生成一个编码二叉树（Huffman树）字符的频率作为权重，这样的Huffman编码既满足前缀编码要求，也能保证总的报文最短。
·在word2vec中，把训练语料中的词当做叶子节点，在语料库出现的次数作为权重。构造相应的Huffman树从而对 

